In [1]:
#imports

import numpy as np
import mxnet as mx
from mxnet import nd, gluon, autograd
import pandas as pd

from sagemaker.mxnet import MXNet
import sagemaker
from sagemaker import get_execution_role

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()
role


'arn:aws:iam::475933981307:role/service-role/AmazonSageMaker-ExecutionRole-20180102T172706'

In [3]:
inputs = sagemaker_session.upload_data(path='data/processed/', bucket='cyrusmv-sagemaker-demos', key_prefix='visa-kaggle/data')
inputs

's3://cyrusmv-sagemaker-demos/visa-kaggle/data'

In [4]:
data_files = {u'train_data': u'data/train_data.npy', 
             u'train_label': u'data/train_label.npy',
             u'val_data': u'data/val_data.npy',
             u'val_label': u'data/val_label.npy'
            }

hyper_parameters = {'batch_size': 100,
                   'learning_rate': 0.01,
                   'num_epochs': 1,
                   'optimizer': 'sgd',
                   'momentum': 0.9,
                   'num_dims': 30}


train_ctx = mx.cpu()
num_examples = 10000
num_input = 30
num_outputs = 2


In [5]:
type(data_files)

dict

In [6]:
!aws s3 ls cyrusmv-sagemaker-demos/visa-kaggle/data/


                           PRE test/
                           PRE train/
2018-01-04 12:19:10          0 


In [7]:
#!cat linear_model_gluon.py

In [13]:
m = MXNet('linear_model_gluon.py', 
          role=role,
          train_instance_count=1,
          train_instance_type="ml.c4.xlarge",
          output_path="s3://cyrusmv-sagemaker-demos/visa-kaggle/output/model",
          hyperparameters=hyper_parameters)

In [14]:
import os
os.listdir('/opt')


['aws']

In [15]:
m.fit(inputs)

INFO:sagemaker:Creating training-job with name: sagemaker-mxnet-py2-cpu-2018-01-05-15-02-46-667


................................................................
executing startup script (first run)
2018-01-05 15:07:59,449 INFO - root - running container entrypoint
2018-01-05 15:07:59,449 INFO - root - starting train task
2018-01-05 15:08:00,857 INFO - mxnet_container.train - MXNetTrainingEnvironment: {'enable_cloudwatch_metrics': False, 'available_gpus': 0, 'channels': {u'training': {u'TrainingInputMode': u'File', u'RecordWrapperType': u'None', u'S3DistributionType': u'FullyReplicated'}}, '_ps_verbose': 0, 'resource_config': {u'current_host': u'algo-1', u'hosts': [u'algo-1']}, 'user_script_name': u'linear_model_gluon.py', 'input_config_dir': '/opt/ml/input/config', 'channel_dirs': {u'training': u'/opt/ml/input/data/training'}, 'code_dir': '/opt/ml/code', 'output_data_dir': '/opt/ml/output/data/', 'output_dir': '/opt/ml/output', 'model_dir': '/opt/ml/model', 'hyperparameters': {u'sagemaker_program': u'linear_model_gluon.py', u'optimizer': u'sgd', u'num_dims': 30, u'sagemaker_regio

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/sagemaker/session.py:630: DeprecationWarning: generator 'multi_stream_iter' raised StopIteration
  for idx, event in sagemaker.logs.multi_stream_iter(client, log_group, stream_names, positions):


Epoch 0, Train_acc ('rmse', 0.0018140961119823974), Test_acc ('rmse', 0.0016063481474329151)




**********************************************************************
saving the model in /opt/ml/model
**********************************************************************




===== Job Complete =====


In [16]:
predictor = m.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-mxnet-py2-cpu-2018-01-05-15-02-46-667
INFO:sagemaker:Creating endpoint with name sagemaker-mxnet-py2-cpu-2018-01-05-15-02-46-667


-----------------------------------------------------------------------------------------------------------------------------!

In [20]:
train_data = np.load('data/processed/train/train_data.npy')
train_label = np.load('data/processed/train/train_label.npy')

test_data = np.load('data/processed/test/val_data.npy')
test_label = np.load('data/processed/test/val_label.npy')

print(train_data.shape, train_label.shape, test_data.shape, test_label.shape)

(170884, 30) (170884,) (113923, 30) (113923,)


In [21]:
predictor.predict(train_data[0:1])

0.0

In [22]:
#since score==1 is very rare we want to make sure we can correctly predict fradulant transaction. 
#First we print a lost of all labels where score == 1, then then run a prediction
t = []
for i in range(len(train_label)):
    if train_label[i] == 1:
        t.append(i)
        
print(t)

[48, 278, 921, 1240, 1295, 1348, 2062, 3238, 3549, 4307, 6305, 6887, 7448, 7558, 7697, 7729, 7752, 9658, 10297, 12178, 13015, 13940, 14306, 15025, 15378, 15843, 15939, 17018, 17454, 17575, 17786, 18699, 19021, 19265, 19306, 19642, 19774, 21181, 21257, 21491, 22530, 24068, 24983, 25715, 25779, 25957, 26155, 27148, 27506, 27832, 28158, 28801, 29014, 29699, 31210, 31237, 31495, 31730, 32247, 32613, 32696, 32787, 33422, 33624, 33651, 34621, 35158, 35159, 35566, 36301, 36634, 37604, 39394, 39840, 40911, 41554, 41799, 41814, 42431, 43028, 43616, 43651, 44184, 44498, 44852, 45296, 45476, 45619, 45887, 46043, 46782, 46938, 46950, 47970, 47977, 48477, 49048, 49207, 49464, 49847, 50313, 50494, 51579, 51689, 52417, 53386, 53440, 53730, 54483, 55461, 55614, 56397, 56452, 56882, 56955, 57901, 57922, 58898, 58902, 58964, 59764, 59847, 61109, 61286, 61833, 62215, 62234, 62595, 63274, 63471, 63768, 64214, 64611, 65408, 66051, 66627, 66720, 66887, 67065, 67573, 67672, 67681, 69141, 69143, 69224, 69519,

In [15]:
hyper_parameters = {'batch_size': 100,
                   'learning_rate': 0.01,
                   'num_epochs': 1,
                   'optimizer': 'sgd',
                   'momentum': 0.9,
                   'num_dims': 30}


In [16]:
import linear_model_gluon as pyfile

In [17]:
pyfile.train({'training': 'data/processed/'}, hyperparameters=hyper_parameters, hosts=['local'], num_gpus=0, output_path="cyrusmv-sagemaker-demos/visa-kaggle/output/model")





**********************************************************************
{'training': 'data/processed/'}
['train', '.ipynb_checkpoints', 'test']
**********************************************************************




(170884, 30) (170884, 30) (113923, 30) (113923, 30)
sequential2_ (
  Parameter sequential2_dense0_weight (shape=(64, 0), dtype=<class 'numpy.float32'>)
  Parameter sequential2_dense0_bias (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter sequential2_dense1_weight (shape=(2, 0), dtype=<class 'numpy.float32'>)
  Parameter sequential2_dense1_bias (shape=(2,), dtype=<class 'numpy.float32'>)
)
Epoch 0, Train_acc ('rmse', 0.0018140961119823974), Test_acc ('rmse', 0.0015975702887037735)


Sequential(
  (0): Dense(64, Activation(relu))
  (1): Dense(2, linear)
)

(170884, 30) (170884,) (113923, 30) (113923,)
